In [24]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [25]:
SEPARATOR = "_"
LABEL_ID = 0
def get_class_names(dataset):
    return set([str(os.path.basename(el.numpy())).split(SEPARATOR)[LABEL_ID] for el in dataset])

In [26]:
def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.argmax(one_hot)

In [27]:
data_path="Input/dataset"

In [28]:
ds = tf.data.Dataset.list_files(f"{data_path}\\*", shuffle=False)
classes = get_class_names(ds)
ds = ds.shuffle(len(ds), reshuffle_each_iteration=False)

In [29]:
classes

{"b'0805S",
 "b'AUTIC",
 "b'C0603",
 "b'C0805",
 "b'C1210",
 "b'CAPACITOR",
 "b'CASEC",
 "b'CONNECTOR",
 "b'CTELRJ45",
 "b'DIGITAL",
 "b'DIODE",
 "b'FUSE",
 "b'GATE",
 "b'INDUCTANCE",
 "b'JUMPER",
 "b'LED",
 "b'LED3",
 "b'MINIMELF",
 "b'MOS",
 "b'OPENFIX",
 "b'OPTOISOLATOR",
 "b'OSCILLATOR",
 "b'PINSMD",
 "b'PINTH",
 "b'PLCC2LED",
 "b'R01005",
 "b'R0402",
 "b'R0603",
 "b'R0805",
 "b'R10",
 "b'R110H126",
 "b'R1206",
 "b'R20",
 "b'R2512",
 "b'R63H40",
 "b'RELAIS",
 "b'RESISTOR",
 "b'S5SOT23",
 "b'SCR",
 "b'SFUSE",
 "b'SMB",
 "b'SO16",
 "b'SO8",
 "b'STRIP3",
 "b'STRIP6",
 "b'TRANSFORMER",
 "b'TRANSISTOR",
 "b'TRIAC",
 "b'VOLTAGEREGULATOR",
 "b'WSK2512",
 "b'X18Y4",
 "b'X43Y55",
 "b'X4Y18",
 "b'X50Y65",
 "b'X63H40",
 "b'X8Y11",
 "b'ZENER"}

In [7]:
val_perc = 0.2
train_ds = ds.skip(int(len(ds)*val_perc))
val_ds = ds.take(int(len(ds)*val_perc))


In [8]:
len(train_ds), len(val_ds)

(60, 14)